In [20]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

df = pd.DataFrame([range(11), range(100,110)], columns=list('abcdefghijk'))

df

,a,b,c,d,e,f,g,h,i,j,k
0,0,1,2,3,4,5,6,7,8,9,10.0
1,100,101,102,103,104,105,106,107,108,109,NaN


In [37]:
# Load Data
model = load_model("myFirstTestModel.h5")
    
with open('MetaData/context_words.pickle', 'rb') as file:
    context_words = pickle.load(file)
with open('MetaData/unique_tokens.pickle', 'rb') as file:
    unique_tokens = pickle.load(file)
with open('MetaData/unique_token_index.pickle', 'rb') as file:
    unique_token_index = pickle.load(file)

In [54]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, context_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
                             
    predictions = model.predict(X,verbose=0)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [55]:
possible = predict_next_word("His that night was",5)

In [56]:
print([unique_tokens[idx] for idx in possible])

['of', 's', 'it', 'the', 'and']


In [57]:
tokenizer = RegexpTokenizer(r"\w+")
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+context_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [58]:
generate_text("His that night was",100,5)

'His that night was s her mother mother her s it it it s a oh it a of a and and of his the and of the of story story in twilight in twilight and of twilight twilight course of you you twilight of twilight in of and story in of twilight of twilight and and in and and of and i of and of story and twilight twilight and the in his in the and the and a of and of story of course twilight of course and story the a his in of course and and the a a his and'